In [1]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import ee
import geemap
import time

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')
print("GEE Authenticated !")

GEE Authenticated !


In [2]:
def gridmet_summary(fires, reducer, attrs, buffer=None):
    """ Calculates the gridMET variables for a given daily perimeter by daily ID 
    Args:
        - fires: fire perimeters (with daily polygons)
        - reducer: the reduction to use for the gridmet variables
        - attrs: the gridmet variables
        - buffer: (optional) apply a buffer to the daily polygon
    """
    # Set a function to calculate the variables for each daily perimeter
    def get_daily_gridmet(perim):
        # Buffer if requested
        if buffer is not None:
            geom = perim.geometry().bounds().buffer(buffer)
        else:
            geom = perim.geometry().bounds()

        # Get the burn date for extracting gridMET variables
        fire_year = ee.Number.parse(perim.get('FIRE_YEAR')) 
        burn_doy = ee.Number.parse(perim.get('ACQ_DOY')) 
        next_doy = burn_doy.add(1)
        
        gridmet_ = gridmet.filter(
            ee.Filter.And(
              ee.Filter.bounds(geom), # filter to the fire bounds
              ee.Filter.calendarRange(fire_year, fire_year, 'year'), # fire year
              ee.Filter.calendarRange(burn_doy, next_doy, 'DAY_OF_YEAR'), # 60 days pre-ignition
            )
        )
        
        gridmet_mn = gridmet_.mean() # take the temporal mean of the imageCollection (daily gridmets)
        
        # Set up the feature reduction
        stats = gridmet_mn.reduceRegion(
            reducer=reducer,
            geometry=geom,
            scale=4000,  # Match gridMET resolution
            bestEffort=True)
        return perim.set(stats)

    # Map the reduction onto the daily polygons
    results = fires.map(get_daily_gridmet)
    
    return results


def monitor_export(task):
    """ Monitors EE export task """
    while task.active():
        print('Waiting for export to finish..\n\tPatience young padawan.')
        time.sleep(30)  # Check every 30 seconds
    
    # Get the status of the task
    status = task.status()
    
    # Check if the task failed or succeeded
    if status['state'] == 'COMPLETED':
        print("Export completed successfully !!!!")
    elif status['state'] == 'FAILED':
        print(f"Export failed! Bummer. Reason: {status.get('error_message', 'Unknown error')}")
    else:
        print(f"Export ended with state: {status['state']}")

print("Functions loaded !")

Functions loaded !


In [ ]:
# Setup the workflow.

In [3]:
# Load the gridMET datasets
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"gridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}")

gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']


In [5]:
fires = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/afd_aspen-fires_2018_to_2023_bounds_daily')
print(f"There are [{fires.size().getInfo()}] daily bounds.")

There are [2037] daily bounds.


In [6]:
# Define the function parameters
reduction = ee.Reducer.percentile([95]) # 95th percentile
attrs = ['fm1000', 'vpd', 'erc', 'bi', 'tmmx', 'rmin', 'vs'] # the gridmet attributes to use
# Retrieve gridmet summaries for daily perimeters
results_fc = gridmet_summary(fires, reduction, attrs)
print("Success !")

Success !


In [7]:
# Check the results
results_fc.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-106.85336123730923, 35.87028325412543],
    [-106.84920075958122, 35.87066463567183],
    [-106.84967784663476, 35.87398984543011],
    [-106.8538385131695, 35.873608467295654],
    [-106.85336123730923, 35.87028325412543]]]},
 'id': '000000000000000000b3',
 'properties': {'ACQ_DOY': 257,
  'FIRE_YEAR': 2018,
  'NIFC_ID': '13378',
  'bi': 41.5,
  'erc': 57,
  'eto': 4.699999809265137,
  'etr': 6.350000381469727,
  'fm100': 6.800000190734863,
  'fm1000': 13.300000190734863,
  'pr': 0,
  'rmax': 43.29999923706055,
  'rmin': 11.25,
  'sph': 0.004414999857544899,
  'srad': 244.14999389648438,
  'th': 196.5,
  'tmmn': 284.1500244140625,
  'tmmx': 298.04998779296875,
  'vpd': 1.7100000381469727,
  'vs': 2.4499998092651367}}

In [8]:
# Export the FeatureCollection to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=results_fc,
    description='afd_aspen-fires_2018_to_2023_bounds_daily-gridmet_stats',
    fileFormat='CSV',
    folder='GRIDMET'
)
# Start the export task
export_task.start()
print("Export to Google Drive started!")
# Monitor the task until it's finished
monitor_export(export_task)

Export to Google Drive started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for expo